<img src="https://relate.cs.illinois.edu/course/cs199py-sp19b/file-version/b9213910148c10426ebdff2bc67ed54051d5ed42/img/quad.png
" width="700px;" alt="quad"/>

#  Scrubbing data and visualizing

## Objectives

### Objectives

1. Retrieve data and place in a pandas dataframe
2. Determine modifications needed in the data 
3. Match the data to visualization package

### Example Questions

1. How can we retrieve data from a webpage?
2. What is a MultiIndex in Pandas?
3. What is a Choropleth map?
4. Can you match some state level data with a map?

## Highlevel topics

- Data retrieval
- Data storage
- Data manipulation
- Visualization

## Synopsis

You're a structural engineer working on a team that analyses the bridge infrastructure in the US.  To make a convincing argument, you are constructing a map of the current bridge conditions across the US.

#### Your Task

Your goal is to plot the bridge conditions at the state level.

## Datasets

In this session two datasets will be used:
- Bridge Condition by Highway System 2019
    - https://www.fhwa.dot.gov/bridge/nbi/no10/condition19.cfm
- Bridge Condition by County 2019
    - https://www.fhwa.dot.gov/bridge/nbi/no10/county19.cfm
- In addition you will use the state/county level geo files:
    - `us-states.json`: https://github.com/python-visualization/folium/tree/master/examples/data
    - http://eric.clst.org/tech/usgeojson/

## Getting Started

We will introduce four new packages in this lesson:

- `requests` is a package the makes URL requests *easy*.  Give it a URL and it retrieves the page.
- `bs4` or BeautifulSoup parses an html file and places it in a convenient structure
- `json` is a package for reading structured JSON files
- `folium` is one of many packages that can be used to plot information on a geographical map

In [1]:
import folium
import os
import pandas as pd
import bs4
import requests
import json

In [38]:
m = folium.Map(
    location=[40.114942, -88.226492],
    #tiles='Stamen Toner',
    tiles="Stamen Terrain",
    zoom_start=13
)

m

## First grab the webpage

Here we'll do two things:

1. retrieve the raw html of the webpage; and
2. parse the html to make a structured `soup`

In [39]:
url = 'https://www.fhwa.dot.gov/bridge/nbi/no10/condition19.cfm'
r = requests.get(url)                  # grab the html source
html = r.text                          # as text
soup = bs4.BeautifulSoup(html, 'lxml') # make a parseable "tree" of html

## <span style='color:red'> ____________________ </span>
We can do any number of things with `soup` at this point.  We can scrub for emails, find links, extract figures, etc.  In this case we wish to find all of the tables in the html.  `<table>` and `<table class="something">` are both examples of tags that we wish to find --- bs4 makes this easy -- try it with `find_all` (https://www.crummy.com/software/BeautifulSoup/bs4/doc/#navigating-using-tag-names)

```
table = ...
```
## <span style='color:red'> Try it! $\downarrow$</span>

## <span style='color:red'> ____________________ </span>
If you have `table`, you can `find_all` on the resulting markup.
```
rows = table.find_all()
```

Use this to find all rows (marked with `tr`) in the table.

## <span style='color:red'> Try it! $\downarrow$</span>

Let's print it:

In [41]:
print(rows[0])
print(rows[1])
print(rows[2])

### One approach

One approach is to zip through the rows, then parse each of the columns.  We may do this like the following:

In [8]:
for row in rows:
    
    # find all 'th' headers
    state_name = row.find('th', {"class": "left"})
    if state_name is not None:
        
        # get the state name
        state_name = state_name.text
        print(state_name)
        
        # get the next four data rows
        count = row.findAll('td')[:4]
        count = [int(c.text.replace(',','')) for c in count]
        print(count)

### Another (easier) approach

## <span style='color:red'> ____________________ </span>
The previous approach is often necessary.  Dirty data, incomplete html, different formats, etc often force us to parse the html by hand.  However in the case of a table, Pandas can be used directly:  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html

How can we use pandas to read the soup table?
## <span style='color:red'> Try it! $\downarrow$</span>

Print the dataframe:

In [44]:
df

Now lets just take the state name and the bridge counts to make the dataframe a bit more manageable:

In [45]:
dfcounts = df[['State','Bridge Counts']].copy()
dfcounts

### A MultiIndex?!

If we take a look a the column names we run into another type in Pandas: a MultiIndex.

In [46]:
dfcounts.columns

One easy thing to do in this case (mainly to make referencing a specific column easier) is to reduce to the column header to a simple `Index`.  Here we'll just use the second level of the MultiIndex:

In [47]:
dfcounts.columns = dfcounts.columns.get_level_values(1)
print(dfcounts.columns)
dfcounts

### Look ahead

Looking ahead to our mapping, we'll be using a GEO file, and each state name will be in the form of `Illinois` or `New Mexico`, etc.  However, in the data frame of bridge data, notice that each state name is in all caps. To fix this, we'll use the `.title()` command and return to lesson1:

In [21]:
'NEW MEXICO'.title()

## <span style='color:red'> ____________________ </span>

Let's use this to change the string in the `State` column:

## <span style='color:red'> Try it! $\downarrow$</span>

Now look at our modified data:

In [49]:
dfcounts

# Your Turn:  Displaying the Data on a Map

Your job is to visualize the information about the bridges on a map (choropleth visualization).  To get you started, this section will go through some examples first of some basic maps with other data.  Go through these examples to get an understanding for the map visualization then complete the 2 tasks at the bottom.

## Example 1:  Our first map

Let's make a map instance with `folium`. Here we set the lat/long coordinates to the Beckman Quad.  The tiles parameter is used to determine they style of the map.  

(You can use `folium.Map?` in this notebook to experiment with different map types -- neat!).  

In [53]:
folium.Map?

In [54]:
m = folium.Map(
    location=[40.114942, -88.226492],
    tiles='Stamen Toner',
    zoom_start=13
)

m

The above example illustrated a map for some specific lat/long coordinates.  In practice, we'll want to display some information on the map.  We'll start in the next example with a simple visualization of data on a map.

## Example 2:  Starting Map with State Data (3 states)

Let's start with a basic visualization of data for 3 states: Iowa, Illinois, and Colorado.  Let's assume we have the following dataframe with some data for a few states. 

In [55]:
df = pd.DataFrame(
    [
        ['Iowa', 'Illinois', 'Colorado'],
        [50, 1, 100]],
    index=['State', 'Some Value']
).T

print(df)

### State JSON Info

To plot on map with Folium, a JSON file is used to describe the polygons that will represent each state.  For example,  Illinois is composed of a list of coordinates -- check it out:

In [56]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'
r = requests.get(url) # grab the source from the url
state_geo = r.json()  # convert to `json`
state_geo['features'][12]

## Just plot it
We could just plot this polygon data on our own:

In [67]:
import matplotlib.pyplot as plt
import numpy as np
coords = state_geo['features'][12]['geometry']['coordinates'][0]
coords = np.array(coords)
plt.plot(coords[:,0], coords[:,1])
plt.axis('equal')

We printed just Illinois above, but  `state_geo` contains the polygons for each of the states.

### Creating Map with State Data

To create a Choropleth, we first initialize a map with folium like we did in the basic example above.  Then, we call `folium.Choropleth()`. 

**What is a Choropleth?** https://en.wikipedia.org/wiki/Choropleth_map

The call to set the `Choropleth` map has 4 important entries:

- `geo_data=state_geo`, here we set the geo data.
- `data=df`, here we set the data *source* (the stuff we'll plot on each state)
- `columns=['State', 'Some Value']`, where to find the state name or state id and the numbers to visualize (in `data`)
- `key_on='feature.properties.name'`, how the entries are represented in the JSON.  For the state geo_data json file, this will be either `'feature.id'` (abbreviation) or `'feature.properties.name'` (name), depending on whether our dataframe uses state names or abbreviations. 

The other parameters control specifics of looks of the visualization (opacity, coloring).

In [71]:
m = folium.Map(location=[44, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=df,
    columns=['State', 'Some Value'],
    key_on='feature.properties.name',
    fill_color='Reds',
    fill_opacity=0.6,
    line_opacity=0.2,
).add_to(m)

folium.LayerControl().add_to(m)

m

## Example 3:  Full Example with Unemployment Data
## <span style='color:red'> ____________________ </span>

For this next part we'll take straight out of the Folium examples.

As before, we use the same json state geo data.  This example reads in unemployment data into a pandas dataframe.  The call to `folium.Choropleth()` is roughly the same as the above, except this dataframe, `state_data`, uses abbreviations not full names, so `key_on=` a different value.  Additionally, the coloring is a little different and the colorbar is labeled.

## <span style='color:red'> Try it! $\downarrow$</span>

In [73]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'
state_geo = requests.get(url).json()

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(url)

m = folium.Map(location=[48, -102], zoom_start=5)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    # vvvv fill this in
    data=None,
    columns=None,
    key_on=None,
    #######
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m


# Your Tasks:
Now that you've seen some examples, it's time for you to visualize the bridge data that is the topic of this lesson.

Your task is to create 2 different choropleth visualizations based on the bridge data (each is described below).

## Task \#1:
## <span style='color:red'> ____________________ </span>

Create a choropleth visualization that illustrates the number of bridges in each state.

Hint:  Think about what the visualization should look like based on the data.  If your visualized results are not as expected, it may help to look at the dataframe to identify what is happening.  Notice any rows that aren't simply state data?
## <span style='color:red'> Try it! $\downarrow$</span>

In [80]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'
state_geo = requests.get(url).json()

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    # fill this in vvvv
    data=None,
    columns=None,
    key_on=None,
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(m)

folium.LayerControl().add_to(m)

m

## Task \#2

## <span style='color:red'> ____________________ </span>
Create a different choropleth visualization that illustrates the percentage of bridges in each state rated as poor.  

Hint:  this value is not currently in the dataframe of bridge data -- you'll need to compute it first

## <span style='color:red'> Try it! $\downarrow$</span>

In [84]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=dfcounts,
    columns=['State', 'Percent in Poor Condition'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Percent of Bridges Ranked as Poor (%)'
).add_to(m)

folium.LayerControl().add_to(m)
m